In [2]:
!pip install transformers
!pip install --upgrade torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 90.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 72.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 12.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 9.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━

In [3]:
! pip install wandb

In [4]:
!pip install scikit-learn

In [5]:
!pip install numpy

In [6]:
import json
import torch
import random
import numpy as np
import torch.nn as nn
import wandb
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from imblearn.over_sampling import RandomOverSampler

In [7]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [8]:
wandb.login(key="9b49f600300d891290c544a9d0580ec7b7185a34")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
wandb.init(project='zaloqa-bert', entity='hdghung2912')

wandb: Currently logged in as: hdghung2912. Use `wandb login --relogin` to force relogin


In [10]:
from sklearn.model_selection import train_test_split

# Load the data from train.json
with open('/kaggle/input/traintest/train.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Split the data into train and test sets (85% train, 15% test)
train_data, test_data = train_test_split(data, test_size=0.15, random_state=42)

# Save the train and test data to separate files
with open('train_split.json', 'w', encoding='utf-8') as train_file:
    json.dump(train_data, train_file, ensure_ascii=False, indent=4)

with open('test_split.json', 'w', encoding='utf-8') as test_file:
    json.dump(test_data, test_file, ensure_ascii=False, indent=4)


In [11]:
# Đọc dữ liệu từ tập tin train_split.json vào biến train_data
with open('train_split.json', 'r', encoding='utf-8') as train_file:
    train_data = json.load(train_file)

# Đọc dữ liệu từ tập tin test_split.json vào biến test_data
with open('test_split.json', 'r', encoding='utf-8') as test_file:
    test_data = json.load(test_file)

In [12]:
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base-v2", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [13]:
def preprocess_data(data, tokenizer):
    max_question_length = 64
    max_text_length = 192
    input_ids = []
    attention_masks = []
    labels = []

    for sample in data:
        question = sample['question']
        text = sample['text']
        label = sample['label']
        
        encoded_question = tokenizer.encode_plus(
            question,
            add_special_tokens=True,
            truncation=True,
            max_length=max_question_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )

        encoded_text = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=max_text_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )

        combined_input_ids = torch.cat((encoded_question['input_ids'], encoded_text['input_ids']), dim=1)
        combined_attention_mask = torch.cat((encoded_question['attention_mask'], encoded_text['attention_mask']), dim=1)

        input_ids.append(combined_input_ids)
        attention_masks.append(combined_attention_mask)
        labels.append(1 if label else 0)

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    print(f"Final input_ids shape: {input_ids.shape}")
    print(f"Final attention_masks shape: {attention_masks.shape}")
    print(f"Final labels shape: {labels.shape}")

    return input_ids, attention_masks, labels


In [14]:
X = [{'question': sample['question'], 'text': sample['text']} for sample in data]
y = [sample['label'] for sample in data]

# Chia dữ liệu thành tập huấn luyện và tập validation (85-15)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

# Tạo một RandomOverSampler
ros = RandomOverSampler(random_state=0)

# Resample tập huấn luyện
X_train_array = np.array([[sample['question'], sample['text']] for sample in X_train])
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_array, y_train)

# Đảm bảo X_train_resampled và y_train_resampled có cùng số lượng mẫu
assert len(X_train_resampled) == len(y_train_resampled)

# In ra số lượng nhãn true và false trong tập huấn luyện sau khi resample
num_true_resampled_train = sum(1 for label in y_train_resampled if label)
num_false_resampled_train = len(y_train_resampled) - num_true_resampled_train
print("Resampled Train - Number of true labels:", num_true_resampled_train)
print("Resampled Train - Number of false labels:", num_false_resampled_train)

Resampled Train - Number of true labels: 10535
Resampled Train - Number of false labels: 10535


In [15]:
resampled_data = [{'question': X_train_resampled[i, 0], 'text': X_train_resampled[i, 1], 'label': y_train_resampled[i]} for i in range(len(X_train_resampled))]

In [16]:
for i in range(5):
    print(f'Question: {X_train_resampled[i, 0]}')
    print(f'Text: {X_train_resampled[i, 1]}')
    print(f'Label: {y_train_resampled[i]}')
    print('---')

Question: Sông Nin đổ ra biển nào
Text: Sông Luỹ đổ ra biển ở cửa biển tại thị trấn Phan Rí
Label: False
---
Question: Tên gọi Nhật Bản nghĩa là gì
Text: Từ ghép này có nghĩa là " nguồn gốc của mặt trời " hoặc " nơi mặt trời mọc " ( từ quan điểm từ Trung Quốc , mặt trời mọc từ phía Nhật Bản ) ; nó là một nguồn cơ sở cho mô tả của phương Tây về Nhật Bản như là " Vùng đất Mặt trời mọc " ( " Land of the Rising Sun " ) .
Label: True
---
Question: Dầu mỏ có màu gì
Text: Loại dầu khoáng này là dầu trong suốt , không màu bao gồm chủ yếu là ankan và cycloankan , liên quan đến thạch dầu mỏ .
Label: False
---
Question: Huyện đảo Phú Quốc có diện tích bao nhiêu
Text: Hồ tiêu Phú Quốc là một loại gia vị được coi là đặc sản của huyện đảo Phú Quốc thuộc Tỉnh Kiên Giang , Việt Nam .
Label: False
---
Question: Cộng hoà Ireland có biên giới trên bộ với quốc gia nào
Text: Ireland là một quốc gia thành viên Liên minh châu Âu từ năm 1973 , song lựa chọn duy trì bên ngoài khu vực Schengen . Công dân Anh Qu

In [17]:
val_data = [{'question': sample['question'], 'text': sample['text'], 'label': label} for sample, label in zip(X_val, y_val)]

In [18]:
for i in range(5):
    print(f'Question: {val_data[i]["question"]}')
    print(f'Text: {val_data[i]["text"]}')
    print(f'Label: {val_data[i]["label"]}')
    print('---')

Question: Lê Lợi với Lê Lai có quan hệ gì
Text: Lê Thái Tổ ở ngôi được 5 năm thì qua đời vào ngày 22 tháng 8 âm lịch (7 tháng 9 dương lịch) năm Quý Sửu (1433), hưởng dương 49 tuổi. Vì nhớ công Lê Lai chết thay cho mình ở núi Chí Linh trước kia, ông dặn lại đời sau phải giỗ Lê Lai trước khi giỗ ông một ngày. Bởi thế đời sau truyền lại câu: "Hăm mốt Lê Lai, hăm hai Lê Lợi."
Label: False
---
Question: ai là phó bí thư hiện tại của Đà Nẵng
Text: Ngày 15 tháng 7 năm 2013 , Thành uỷ Đà Nẵng công bố quyết định số 7340 / QĐ-TU thành lập Ban Nội chính Thành uỷ Đà Nẵng . Ông Trần Thanh Vân , lúc này là Thành uỷ viên Thành uỷ Đà Nẵng , Viện trưởng Viện kiểm sát nhân dân thành phố Đà Nẵng , được điều động , bổ nhiệm làm Trưởng Ban Nội chính Thành uỷ Đà Nẵng nhiệm kì 5 năm , hai Phó ban là Nhật Thành , Phó Trưởng Ban thường trực Ban chỉ đạo Phòng chống tham nhũng thành phố Đà Nẵng và ông Phạm Hà Bắc , Phó Trưởng Ban chỉ đạo phòng chống tham nhũng thành phố Đà Nẵng .
Label: False
---
Question: Ai là

In [19]:
train_input_ids, train_attention_masks, train_labels = preprocess_data(resampled_data, tokenizer)
val_input_ids, val_attention_masks, val_labels = preprocess_data(val_data, tokenizer)

Final input_ids shape: torch.Size([21070, 256])
Final attention_masks shape: torch.Size([21070, 256])
Final labels shape: torch.Size([21070])
Final input_ids shape: torch.Size([2717, 256])
Final attention_masks shape: torch.Size([2717, 256])
Final labels shape: torch.Size([2717])


In [20]:
# Tạo TensorDataset cho tập huấn luyện
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)

# Tạo DataLoader cho tập huấn luyện
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=8)

# Tạo TensorDataset cho tập validation
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)

# Tạo DataLoader cho tập validation
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [21]:
for batch in train_dataloader:
    batch_input_ids, batch_attention_masks, batch_labels = batch
    print(f"Batch input_ids shape: {batch_input_ids.shape}")
    print(f"Batch attention_masks shape: {batch_attention_masks.shape}")
    print(f"Batch labels shape: {batch_labels.shape}")
    break

Batch input_ids shape: torch.Size([8, 256])
Batch attention_masks shape: torch.Size([8, 256])
Batch labels shape: torch.Size([8])


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [23]:
from torch.optim import Adam
optimizer = Adam(model.parameters(), lr=2e-5)
criterion = nn.BCEWithLogitsLoss()

In [24]:
epochs = 8
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [25]:
def train(model, train_dataloader, epochs, optimizer, scheduler, device):
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        total_loss = 0
        correct_preds = 0
        total_preds = 0

        # Sử dụng tqdm để hiển thị tiến trình huấn luyện
        for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
            batch_input_ids, batch_attention_masks, batch_labels = tuple(t.to(device) for t in batch)

            model.zero_grad()
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            # Tính accuracy
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct_preds += torch.sum(preds == batch_labels).item()
            total_preds += len(batch_labels)

        avg_train_loss = total_loss / len(train_dataloader)
        train_accuracy = correct_preds / total_preds
        
        # Ghi loss và accuracy vào Weights & Biases
        wandb.log({"Train Loss": avg_train_loss, "Train Accuracy": train_accuracy})

        print(f"Epoch {epoch + 1}, Average Training Loss: {avg_train_loss}, Train Accuracy: {train_accuracy}")

In [26]:
train(model, train_dataloader, epochs, optimizer, scheduler, device)

Epoch 1/8


Training: 100%|██████████| 2634/2634 [15:57<00:00,  2.75it/s]


Epoch 1, Average Training Loss: 0.4649982833824684, Train Accuracy: 0.796298054105363
Epoch 2/8


Training: 100%|██████████| 2634/2634 [16:01<00:00,  2.74it/s]


Epoch 2, Average Training Loss: 0.2826737343710575, Train Accuracy: 0.9172282866635026
Epoch 3/8


...

Epoch 6, Average Training Loss: 0.10989269017743622, Train Accuracy: 0.9747982914095871
Epoch 7/8


Training: 100%|██████████| 2634/2634 [16:03<00:00,  2.73it/s]


Epoch 7, Average Training Loss: 0.10596689911634889, Train Accuracy: 0.9761271950640722
Epoch 8/8


Training: 100%|██████████| 2634/2634 [16:02<00:00,  2.74it/s]

Epoch 8, Average Training Loss: 0.10646393136372373, Train Accuracy: 0.9756525866160418


In [30]:
def evaluate_model(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in dataloader:
            batch_input_ids, batch_attention_masks, batch_labels = tuple(t.to(device) for t in batch)

            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks)
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=1).tolist())
            true_labels.extend(batch_labels.tolist())

    predictions = np.array(predictions)
    true_labels = np.array(true_labels)

    f1 = f1_score(true_labels, predictions)
    cm = confusion_matrix(true_labels, predictions)

    return f1, cm

In [31]:
evaluate_model(model, val_dataloader, device)

(0.78703186137507,
 array([[1632,  203],
        [ 178,  704]]))

In [33]:
checkpoint_path = "./results/checkpoint"
model.save_pretrained(checkpoint_path)
tokenizer.save_pretrained(checkpoint_path)

('./results/checkpoint/tokenizer_config.json',
 './results/checkpoint/special_tokens_map.json',
 './results/checkpoint/vocab.txt',
 './results/checkpoint/bpe.codes',
 './results/checkpoint/added_tokens.json')